In [15]:
import sys
sys.path.append('/Users/diegodavila/miniconda3/lib/python3.7/site-packages')
import flywheel
import nibabel as nb
import os

fw = flywheel.Client()
projectContainer = fw.projects.find("label=GRMPY_822831")
projectContainer = projectContainer[0]
sessions = projectContainer.sessions()
subjects = projectContainer.subjects()

/Users/diegodavila/miniconda3/lib/python3.7/site-packages/flywheel/flywheel.py:5758: UserWarning: Client version 9.0.3 does not match server version 10.7.3. Please update your client version!
  warnings.warn('Client version {} does not match server version {}. Please update your client version!'.format(SDK_VERSION, release_version))


In [101]:
def identify_dicomref(acq):
    
    asl_to_return = None
    is_asl = False
    
    acq = fw.get(acq.id)
    filelist = [x for x in acq.files if x.type in ["nifti"]]
    
    for f in filelist:
        if "dicomref" in f.name:
            asl_to_return = f
            is_asl = True
        else:
            continue
                        
    if is_asl:
        return asl_to_return

In [99]:
sessions[177].acquisitions()[12].files[4].name

'12.dicom.zip'

In [102]:
asl_list = [identify_dicomref(x) for x in sessions[177].acquisitions()]
asl_list

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 {'classification': {'Measurement': ['Perfusion'], 'measurement': ['Perfusion']},
  'created': datetime.datetime(2019, 2, 27, 8, 0, 47, 14000, tzinfo=tzutc()),
  'hash': 'v0-sha384-1e62c1861bfa7f78b7bd5f5ae316dabf0a023c09f02fcaf860c6c9fe87503cde45dbba385341ae278274ba65bd2804fd',
  'id': '57f4b610-788e-4331-8c10-203d42547ae3',
  'info': {'AcquisitionDateTime': '2016-05-26T12:51:32.067500',
           'AcquisitionMatrixPE': 64,
           'AcquisitionNumber': 1,
           'AcquisitionTime': '12:51:32.067500',
           'BIDS': {'Acq': 'ref',
                    'Ce': '',
                    'Dir': '',
                    'Echo': '',
                    'Filename': 'sub-88608_ses-10165_acq-ref_asl.nii.gz',
                    'Folder': 'asl',
                    'IntendedFor': '',
                    'Mod': '',
                    'Modality': '',
                    'Path': 'sub-88608/ses-10165/asl',
   

In [103]:
def get_asl(sess):
    acs = sess.acquisitions()
    asl_list = [identify_dicomref(x) for x in acs]
    asl_list = [i for i in asl_list if i]
    if len(asl_list) >= 1:
        return asl_list[0]
    else:
        return None

In [108]:
q = get_asl(sessions[177])
if q != None:
    sessions[177].download_file(q.name,"")

{'classification': {'Measurement': ['Perfusion'], 'measurement': ['Perfusion']},
 'created': datetime.datetime(2019, 2, 27, 8, 0, 47, 14000, tzinfo=tzutc()),
 'hash': 'v0-sha384-1e62c1861bfa7f78b7bd5f5ae316dabf0a023c09f02fcaf860c6c9fe87503cde45dbba385341ae278274ba65bd2804fd',
 'id': '57f4b610-788e-4331-8c10-203d42547ae3',
 'info': {'AcquisitionDateTime': '2016-05-26T12:51:32.067500',
          'AcquisitionMatrixPE': 64,
          'AcquisitionNumber': 1,
          'AcquisitionTime': '12:51:32.067500',
          'BIDS': {'Acq': 'ref',
                   'Ce': '',
                   'Dir': '',
                   'Echo': '',
                   'Filename': 'sub-88608_ses-10165_acq-ref_asl.nii.gz',
                   'Folder': 'asl',
                   'IntendedFor': '',
                   'Mod': '',
                   'Modality': '',
                   'Path': 'sub-88608/ses-10165/asl',
                   'Rec': '',
                   'Run': '',
                   'Task': '',
              

In [1]:

file=nb.load('86924_20160119x10035.nii.gz')

/Users/adebimpe/opt/miniconda3/lib/python3.7/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [2]:
file

In [3]:
data1=file.get_data()
data1.shape

(64, 64, 32, 82)

In [23]:
asl=data1[:,:,:,0:80]
m0=data1[:,:,:,80:82]

In [25]:
aslfile=nb.Nifti1Image(dataobj=asl,header=file.header,affine=file.affine)
mofile=nb.Nifti1Image(dataobj=m0,header=file.header,affine=file.affine)

In [26]:
nb.save(aslfile,filename='aslfile.nii.gz')
nb.save(mofile,filename='m0.nii.gz')